In [5]:
from pathlib import Path
import pandas
import numpy as np

def process_csv(input_path, output_path):
    df = pandas.read_csv(input_path)
    df = df.drop('ts', axis=1).copy()
    last_timestamp = df['t_s'].max()
    recording_start_approx = last_timestamp - 30  # buffer for ~30s recording
    # Look at data around that time
    window = df[(df['t_s'] >= recording_start_approx - 15) & 
                (df['t_s'] <= recording_start_approx + 15)]
    df['t_s'] = df['t_s'] - (window['t_s'].iloc[0])
    df = df[df['t_s'] >= 0]
    df["Time (s)"] = df["t_s"].round(2)
    features_df = (df.pivot_table(
            index="Time (s)",
            columns="counter_name",
            values="value",
            aggfunc="mean", # if multiple rows have same counter_name and time bin, mean value is used
            observed=True)
            .sort_index())
    features_df = features_df.reset_index()
    features_df.columns.name = None
    features_df = features_df.replace(0, np.nan)
    features_df = features_df.dropna(axis=1, how='all')
    features_df = features_df.ffill()
    features_df = features_df.bfill()
    features_df = features_df.dropna(axis=1, how='any')
    features_df = features_df[features_df['Time (s)'] >= 7]
    if features_df.shape[1] >= 50:
        features_df.to_csv(output_path, index=False)

to_clean = Path("meta_scan_csvs/uncleaned/post_update")
for csv_file in to_clean.rglob("*.csv"):
    output_file = Path(f"{csv_file.stem}_cleaned.csv")
    print(f"Processing {csv_file.name}...")
    process_csv(csv_file, output_file)
    print(f"Created {output_file.name}\n")

Processing blinds_10.csv...
Created blinds_10_cleaned.csv

Processing blinds_6.csv...
Created blinds_6_cleaned.csv

Processing blinds_7.csv...
Created blinds_7_cleaned.csv

Processing blinds_8.csv...
Created blinds_8_cleaned.csv

Processing blinds_9.csv...
Created blinds_9_cleaned.csv

Processing blinds_up_10.csv...
Created blinds_up_10_cleaned.csv

Processing blinds_up_6.csv...
Created blinds_up_6_cleaned.csv

Processing blinds_up_7.csv...
Created blinds_up_7_cleaned.csv

Processing blinds_up_8.csv...
Created blinds_up_8_cleaned.csv

Processing blinds_up_9.csv...
Created blinds_up_9_cleaned.csv

Processing blinds_up_motion_10.csv...
Created blinds_up_motion_10_cleaned.csv

Processing blinds_up_motion_6.csv...
Created blinds_up_motion_6_cleaned.csv

Processing blinds_up_motion_7.csv...
Created blinds_up_motion_7_cleaned.csv

Processing blinds_up_motion_8.csv...
Created blinds_up_motion_8_cleaned.csv

Processing blinds_up_motion_9.csv...
Created blinds_up_motion_9_cleaned.csv

Processin